# Load weights from TF model

*Note: A kernel with tf 1 is needed*

### Download pre-trained models

In [20]:
# from https://github.com/tensorflow/tpu/blob/master/models/official/mnasnet/mnasnet_example.ipynb

checkpoint_name = 'mnasnet-a1' #@param
url = 'https://storage.googleapis.com/mnasnet/checkpoints/' + checkpoint_name + '.tar.gz'
print('Downloading from ', url)

!wget -nc {url}
print('Unpacking')
!tar -xvf {checkpoint_name}.tar.gz

display.clear_output()
print('Successfully downloaded checkpoint from ', url,
      '. It is available as', checkpoint_name)

Successfully downloaded checkpoint from  https://storage.googleapis.com/mnasnet/checkpoints/mnasnet-a1.tar.gz . It is available as mnasnet-a1


### Load model

In [21]:
import os
import tensorflow as tf

export_dir = os.path.join(checkpoint_name, 'saved_model')

with tf.Graph().as_default() as graph, tf.Session() as sess:
    tf.saved_model.loader.load(sess, [tf.saved_model.tag_constants.SERVING], export_dir)

    vars_global = tf.global_variables()
    model_vars = {}    
    # get their name and value and put them into dictionary
    sess.as_default()
    for var in vars_global:
        try:
            model_vars[var.name] = var.eval()
        except:
            print("For var={}, an exception occurred".format(var.name))

AttributeError: module 'tensorflow' has no attribute 'Session'

### Save weights

In [40]:
import numpy as np

np.save("tf_weights.npy", model_vars)
!rm -rf mnasnet-a1

# Transfer weights to TF2 keras model

*Note: A kernel with tf 2 is needed*

In [22]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [23]:
import numpy as np

tf_weights = np.load('tf_weights.npy', allow_pickle=True).item()

FileNotFoundError: [Errno 2] No such file or directory: 'tf_weights.npy'

In [24]:
from MnasNet_Functional import build_mnasnet_model

model = build_mnasnet_model('mnasnet-a1')

In [25]:
def verify_and_set(layer, block_id, tf_weights, tf_keys, curr_index, verbose):
    name = layer.name
    start = 'mnasnet-a1'
    block_name = f'mnas_blocks_{block_id}'
    
    if name.endswith('depthwise_conv'):
        key = '{}/mnas_net_model/{}/depthwise_conv2d'.format(start, block_name)
        forward = 1
    
    elif name.endswith('se_reduce_conv') or name.endswith('se_expand_conv'):
        key = '{}/mnas_net_model/{}/se/conv2d'.format(start, block_name)
        forward = 2
    
    elif name.endswith('stem_conv') or name.endswith('head_conv'):
        key = '{}/mnas_net_model/mnas_{}/conv2d'.format(start, name[:4])
        forward = 1
    
    elif name.endswith('stem_conv'):
        key = '{}/mnas_net_model/mnas_stem/conv2d'.format(start)
        forward = 1
    
    elif name.endswith('conv'):
        key = '{}/mnas_net_model/{}/conv2d'.format(start, block_name)
        forward = 1

    elif name.endswith('stem_conv_BN') or name.endswith('head_conv_BN'):
        key = '{}/mnas_{}/batch_normalization'.format(start, name[:4])
        forward = 4

    elif name.endswith('BN'):
        key = '{}/{}/batch_normalization'.format(start, block_name)
        forward = 4
    
    elif name == 'FC':
        key = '{}/mnas_net_model/mnas_head/dense'.format(start)
        forward = 2

    else:
        if layer.variables != []:
            raise ValueError(f'Layer "{name}" is not supported')
        return curr_index
    
    weights = []
    for i in range(curr_index, curr_index+forward):
        if not tf_keys[i].startswith(key):
            msg = 'For layer={}, an exception occurred\n'
            msg += "\ttf_index:\t{}\n\taccess_key:\t{}\n\treal_key:\t{}"
            raise ValueError(msg.format(name, i, key, tf_keys[i]))
        weights.append(tf_weights[tf_keys[i]])
    layer.set_weights(weights)  

    if verbose:
        print(f'Processesd "{name}"')
    return curr_index+forward


def keras_set_weights_from_tf_model(model, tf_weights, verbose=False):
    tf_index = 1
    block_id = 0
    b_chars = ('0',) * 2
    tf_keys = list(tf_weights.keys())
    
    for layer in model.layers:
        name = layer.name
        if name.startswith('block'):
            if b_chars != (name[6], name[12]):
                block_id += 1
                b_chars = (name[6], name[12])
        
        tf_index = verify_and_set(
            layer,
            block_id,
            tf_weights,
            tf_keys,
            tf_index,
            verbose
            )      

In [26]:
keras_set_weights_from_tf_model(model, tf_weights)

In [35]:
model.save_weights('MnasNet_tf2_keras.h5')

# Evaluate

In [28]:
!wget -nc -q https://upload.wikimedia.org/wikipedia/commons/f/fe/Giant_Panda_in_Beijing_Zoo_1.JPG -O panda.jpg

In [37]:
from IPython import display
import pylab
import PIL
import numpy as np
filename = 'panda.jpg'
img = np.array(PIL.Image.open(filename).resize((224, 224))).astype(np.float)

In [38]:
import common.imagenet as imagenet
from scipy.special import softmax
from MnasNet_Functional import build_mnasnet_model


model = build_mnasnet_model('mnasnet-a1')
model.load_weights('MnasNet_tf2_keras.h5')

logits = model.predict(img[np.newaxis,...])

top_class = np.argmax(logits)
probs = softmax(logits)

print("Top class: ", top_class, " with Probability= ", probs[0][top_class])
label_map = imagenet.create_readable_names_for_imagenet_labels()  
for idx, label_id in enumerate(reversed(list(np.argsort(probs)[0][-5:]))):
    print("Top %d Prediction: %d, %s, probs=%f" % (idx+1, label_id, label_map[label_id], probs[0][label_id]))

Top class:  388  with Probability=  0.8776357
Top 1 Prediction: 388, lesser panda, red panda, panda, bear cat, cat bear, Ailurus fulgens, probs=0.877636
Top 2 Prediction: 245, Tibetan mastiff, probs=0.002865
Top 3 Prediction: 384, Madagascar cat, ring-tailed lemur, Lemur catta, probs=0.002584
Top 4 Prediction: 296, American black bear, black bear, Ursus americanus, Euarctos americanus, probs=0.001733
Top 5 Prediction: 222, Irish water spaniel, probs=0.001599


# Convert to TFLite

In [5]:
# Load the MnasNet tf.keras model.
from MnasNet_Functional import build_mnasnet_model

model = build_mnasnet_model('mnasnet-a1')
model.load_weights('MnasNet_tf2_keras.h5')

In [2]:
import numpy as np
import tensorflow as tf

# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the TensorFlow Lite model on random input data.
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
tflite_results = interpreter.get_tensor(output_details[0]['index'])

# Test the TensorFlow model on random input data.
tf_results = model(tf.constant(input_data))

# Compare the result.
for tf_result, tflite_result in zip(tf_results, tflite_results):
    np.testing.assert_almost_equal(tf_result, tflite_result, decimal=5)

In [6]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("MnasNet.tflite", "wb").write(tflite_model)

15515740

In [4]:
model = build_mnasnet_model('mnasnet-a1',
                            dict(normalize_input=False))
model.load_weights('MnasNet_tf2_keras.h5')

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_model = converter.convert()
open("MnasNet_quant.tflite", "wb").write(tflite_model)

3983064